<a href="https://colab.research.google.com/github/vanikanjirangat/BioRelation-Extraction-BERT-SDP-Triplets/blob/master/CMED_clinical_idx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
root_dir = "/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/"

In [ ]:
root_dir = "/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/"
import os
input_dir=os.path.join(root_dir,'train')

In [ ]:
input_dir

'/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/train'

In [ ]:
root_dir = "/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/"
import os
input_dir1=os.path.join(root_dir,'dev')

In [ ]:
import glob
import re
items=[]
c1=glob.glob(input_dir1+"/*.ann")
for c in c1:
  with open(c) as f1:
    p=c.split(".")[0]
    m=f1.read()
    m1=m.split("\n")
    m2=[k.split("\t") for k in m1]
    m3=[(k[0],k[1],k[-1]) for k in m2 if len(k)==3]
    spans=[re.findall('\d+', s[1]) for s in m3]
    labels=[re.split('\d+', s[1])[0] for s in m3]
    m4=[(k[0],labels[i],spans[i],k[-1]) for i,k in enumerate(m3)]
    items.append((p,m4))

In [ ]:
txtfiles=[]
c2=glob.glob(input_dir1+"/*.txt")

for c in c2:
  with open(c) as f1:
    p=c.split(".")[0]
    text=f1.read()
    txtfiles.append((p,text))

In [ ]:
print(len(txtfiles))

50


In [ ]:
len(items)

50

In [ ]:
data=[]
textdata=[]
ptext=[g[0] for g in txtfiles]
for i,item in enumerate(items):
  k={}
  #print(i)
  path=item[0]
  #print(path)
  if path in ptext:
    idx=ptext.index(path)
  #print(idx) 
  #print(ptext[idx])
  text=txtfiles[idx][1]
  textdata.append(text)

In [ ]:
len(textdata)

50

In [ ]:
data=[]
#textdata=[]
ptext=[g[0] for g in txtfiles]
for i,item in enumerate(items):
  k={}
  #print(i)
  path=item[0]
  #print(path)
  if path in ptext:
    idx=ptext.index(path)
  doc_id=path.split("/")[-1]
  #print(idx) 
  #print(ptext[idx])
  text=txtfiles[idx][1]
  doc = nlp(text) # use your raw text here
  for x in item[1]:
    define_words=x[-1]
    
    
    span=int(x[2][0]),int(x[2][1])
    label=x[1]
    w=x[-1]
    word = doc.char_span(span[0], span[1])
    #if str(word)!=define_words:
      #print((idx,word,define_words))
    if word is not None:
      sent = word.sent
    else:
      sent=re.findall(r"([^\n\n]*?%s.*?.[$\n\n])" % define_words,text)
      if sent==[]:
        sent=define_words
      #print("sent",sent)
    
    #text=text.replace(define_words,"$"+text[span[0]:span[1]]+"$")
    #t=[sentence + '.' for sentence in textdata[i].split('.') if define_words in sentence]
    #t=(re.findall(r"([^\n\n]*?%s.*?.[$\n\n])" % define_words,text))
    #t=(re.findall(r"([^\n\n]*?%s.*?.[$\n\n])" % define_words,text))
    #print(t)
    data.append((sent,define_words,span,label,doc_id))
    #if x[-1] not in k.keys():
      #k[x[-1]]=t
  #data.append(k)
  #textdata.append(text)

In [ ]:
import pandas as pd
df = pd.DataFrame(data,columns=["sent","word","span","label","doc_id"])

In [ ]:
import pandas as pd
df_dev = pd.DataFrame(data,columns=["sent","word","span","label","doc_id"])

In [ ]:
df_dev.head()

,Unnamed: 0,sent,word,span,label
0,0,"Pt says he can't stop eating after 3-4 pm, get...",barbiturate,"(767, 778)",Disposition
1,1,"Pt says he can't stop eating after 3-4 pm, get...",Ativan,"(783, 789)",Disposition
2,2,"Started Tramadol, thinks it's helping curb his...",Tramadol,"(1951, 1959)",Disposition
3,3,Highest weight: 220->255 (2103 dx'd with CNS ...,prednisone,"(2186, 2196)",Disposition
4,4,Gained 35# on Prednisone x 4 yrs.,Prednisone,"(2491, 2501)",NoDisposition


In [ ]:
df.to_csv(input_dir+"/train.csv")

In [ ]:
df_dev.to_csv(input_dir+"/dev.csv")

In [ ]:
df_train=pd.read_csv(input_dir+"/train.csv")

In [ ]:
df_train.head()

,Unnamed: 0,sent,word,span,label,doc_id
0,0,"At the time of our last visit, she \n\nbegan P...",Procardia XL,"(119, 131)",Disposition,228-03
1,1,"After being on the dose \n\nof Procardia, she ...",Procardia,"(264, 273)",Disposition,228-03
2,2,Her other medications have remained the same a...,Nitroglycerin,"(766, 779)",Disposition,228-03
3,3,Her other medications have remained the same a...,Nifedipine,"(751, 761)",Disposition,228-03
4,4,Change Procardia XL to 60 mg once daily and co...,Procardia XL,"(1067, 1079)",Disposition,228-03


In [ ]:
# d=[]
# for j,item in enumerate(items):
#   print("item no:",j)
#   k=0
#   word1=[]
#   i=0
#   for x in item[1]:
#     k+=1
#     #('T1', 'NoDisposition ', ['619', '625'], 'Nexium')
#     print(x)
#     word=x[-1]
#     label=x[1]
#     span=x[2]
    
    
#     if word in data[j].keys():
      
#       if word in word1:
#         i+=1
#       else:
#         i=0
#       print(i)
#       sent=data[j][word]
#       sents=sent[i]
#       word1.append(word)
#       print(sents)
#       #print((k,sents,word,label))
      
#       d.append((sent,word,span,label))

**Training & Testing**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
root_dir = "/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/"
import os
input_dir=os.path.join(root_dir,'train')

In [ ]:
import pandas as pd
df=pd.read_csv(root_dir+"/train.csv")

In [ ]:
df.head()

,Unnamed: 0,sent,word,span,label,doc_id
0,0,"At the time of our last visit, she \n\nbegan P...",Procardia XL,"(119, 131)",Disposition,228-03
1,1,"After being on the dose \n\nof Procardia, she ...",Procardia,"(264, 273)",Disposition,228-03
2,2,Her other medications have remained the same a...,Nitroglycerin,"(766, 779)",Disposition,228-03
3,3,Her other medications have remained the same a...,Nifedipine,"(751, 761)",Disposition,228-03
4,4,Change Procardia XL to 60 mg once daily and co...,Procardia XL,"(1067, 1079)",Disposition,228-03


In [ ]:
df_dev=pd.read_csv(root_dir+"/dev.csv")

In [ ]:
df_dev.head()

,Unnamed: 0,sent,word,span,label,doc_id
0,0,"Pt says he can't stop eating after 3-4 pm, get...",barbiturate,"(767, 778)",Disposition,187-03
1,1,"Pt says he can't stop eating after 3-4 pm, get...",Ativan,"(783, 789)",Disposition,187-03
2,2,"Started Tramadol, thinks it's helping curb his...",Tramadol,"(1951, 1959)",Disposition,187-03
3,3,Highest weight: 220->255 (2103 dx'd with CNS ...,prednisone,"(2186, 2196)",Disposition,187-03
4,4,Gained 35# on Prednisone x 4 yrs.,Prednisone,"(2491, 2501)",NoDisposition,187-03


In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
le = LabelEncoder()


Found GPU at: /device:GPU:0


In [ ]:
'''
Path to biobert checkpoints, which are already converted to pytorch
'''
root_dir1 = "/content/gdrive/My Drive/"
path_=os.path.join(root_dir1,'biobert_T')
#VOCAB_FILE=path_+'/vocab.txt'
#print(VOCAB_FILE)
#tokenizer = BertTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoConfig,AutoModelForSequenceClassification
class Model:
    def __init__(self,path,f1=0):
        # self.args = args
        self.f1=f1
        print("flag status:(1:sentence pair, 0: single sentence)",f1)
        self.path=path
        self.MAX_LEN=128
        self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        num_labels=3
        self.config = AutoConfig.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=num_labels)

       
        # if not os.path.isdir(self.opath):
        #     os.makedirs(self.opath)
            
            
    def extract_data(self,name):
        file =self.path+name
        df = pd.read_csv(file)
        #print(df.head())
        df.replace(np.nan,'NIL', inplace=True)
        
        sentences = df.sent.values
        entity=df.word.values
        labels = df.label.values
        span=df.span.values
        doc_id=df.doc_id.values
        
        
        
        return (sentences,entity,labels,doc_id)

    def process_inputs(self,e,sentences,labels):
      #entity=[ast.literal_eval(x) for x in e]
      #triplets=[ast.literal_eval(x) for x in t]
      #triplets_sents=[('_'.join(x[0:2]),'_'.join(x[2:4])) if len(x)==4 else ('_'.join(x[0:2]),'_'.join(x[1:3])) for x in triplets]
      if self.f1==1:
        sentences = [self.tokenizer.encode_plus(sent,e[i],add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]#sentence-pair
      else:
        sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]#single sentence
      
      tags_vals = list(labels)

      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      print(le_name_mapping)
      labels=le.fit_transform(labels)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

      if self.f1==1:
        token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
        token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

        inputs, labels,types = input_ids, labels,token_type_ids
      else:
        inputs,labels=input_ids, labels

      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      if self.f1==1:
        self.types=torch.tensor(types).to(torch.int64)
        self.data = TensorDataset(self.inputs, self.types,self.masks, self.labels)
      else:
        self.data = TensorDataset(self.inputs, self.masks, self.labels)

      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=32)

    def process_inputs_test(self,e,sentences,labels,act_ids,batch_size=1):
      #entity=[ast.literal_eval(x) for x in e]
      #triplets=[ast.literal_eval(x) for x in t]
      #triplets_sents=[('_'.join(x[0:2]),'_'.join(x[2:4])) if len(x)==4 else ('_'.join(x[0:2]),'_'.join(x[1:3])) for x in triplets]
      if self.f1==1:
        sentences = [self.tokenizer.encode_plus(sent,e[i],add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]#sentence-pair
      else:
        sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]#single sentence
      sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      torch_idx = torch.tensor(sentence_idx)
      tags_vals = list(labels)

      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      print(le_name_mapping)
      labels=le.fit_transform(labels)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

      if self.f1==1:
        token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
        token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

        inputs, labels,types = input_ids, labels,token_type_ids
      else:
        inputs,labels=input_ids, labels

      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      self.torch_idx = torch.tensor(sentence_idx).to(torch.int64)
      self.act_ids = torch.tensor(act_ids).to(torch.int64)
        #self.words=torch.tensor(e).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      
      if self.f1==1:
        self.types=torch.tensor(types).to(torch.int64)
        self.data = TensorDataset(self.inputs, self.types,self.masks, self.labels,self.torch_idx,self.act_ids)
      else:
        self.data = TensorDataset(self.inputs, self.masks, self.labels,self.torch_idx,self.act_ids)

      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=batch_size)

      # return (self.inputs,self.labels,self.masks,self.types)
    def train_save_load(self,path_,train=1):
      self.model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",config=self.config)
     
      #self.model = BertForSequenceClassification.from_pretrained(path_, num_labels=3)
      self.model.cuda()
      param_optimizer = list(self.model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},{'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                                                                                                                                     'weight_decay_rate': 0.0}]
      #optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)
      optimizer = torch.optim.AdamW(optimizer_grouped_parameters,lr=2e-5)
      
      WEIGHTS_NAME = "CMED_clinicalpair.bin"
      OUTPUT_DIR = root_dir+"/models/"
      output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
      train_loss_set = []
      epochs = 4
      import time
      start_time = time.time()
      if train==1:
        for _ in trange(epochs, desc="Epoch"):
          # Trainin
          # Set our model to training mode (as opposed to evaluation mode
          self.model.train()
          # Tracking variables
          tr_loss = 0
          nb_tr_examples, nb_tr_steps = 0, 0
          # Train the data for one epoch
          for step, batch in enumerate(self.dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            if self.f1==1:
              b_input_ids, b_token_type,b_input_mask, b_labels = batch
            else:
              b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            # loss = model(b_input_ids, token_type_ids=b_types, attention_mask=b_input_mask, labels=b_labels)
            if self.f1==1:
              loss,logits= self.model(b_input_ids, token_type_ids=b_token_type, attention_mask=b_input_mask, labels=b_labels,return_dict=False)
            else:
              loss,logits= self.model(b_input_ids, attention_mask=b_input_mask, labels=b_labels,return_dict=False)

            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
          print("Train loss: {}".format(tr_loss/nb_tr_steps))
        print("--- %s seconds ---" % (time.time() - start_time)) 
        torch.save(self.model.state_dict(), output_model_file)
      else:
        state_dict = torch.load(output_model_file)
        self.model.load_state_dict(state_dict) 
    def test(self):
      # Put model in evaluation mod
      self.model.eval()
      # Tracking variables 
      self.predictions , self.true_labels = [], []
      # Predict 
      for batch in self.dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        if self.f1==1:
          b_input_ids,b_type, b_input_mask, b_labels = batch
        else:
          b_input_ids,b_input_mask, b_labels = batch
        # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          if self.f1==1:
            outputs = self.model(b_input_ids, token_type_ids=b_type, attention_mask=b_input_mask)
          else:
            outputs = self.model(b_input_ids, attention_mask=b_input_mask)
        logits=outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Store predictions and true labels
        self.predictions.append(logits)
        self.true_labels.append(label_ids)
    def compute(self):
      flat_true_labels1=[]
      # Flatten the predictions and true values 
      flat_predictions = [item for sublist in self.predictions for item in sublist]
      flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
      flat_true_labels = [item for sublist in self.true_labels for item in sublist]
      print('Classification Report')
      labels_test=flat_predictions
      print(classification_report(flat_true_labels,flat_predictions))
      print(confusion_matrix(flat_true_labels,flat_predictions))

    def test_id(self,sents,words,doc_id):
        batch_size=32
        # Put model in evaluation mod
        self.model.eval()
        # Tracking variables 
        self.predictions ,self.true_labels,self.sents,self.actsents = [], [],[],[]
        output_dicts=[]
      
        
        for batch in self.dataloader:
          # Add batch to GPU
          batch = tuple(t.to(device) for t in batch)
          # Unpack the inputs from our dataloader
          b_input_ids,b_type,b_input_mask, b_labels,b_index,b_ids = batch
          # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
          with torch.no_grad():
            # Forward pass, calculate logit predictions
            if self.f1==1:
              outputs = self.model(b_input_ids, token_type_ids=b_type, attention_mask=b_input_mask)
            else:
              outputs = self.model(b_input_ids, attention_mask=b_input_mask)
            logits=outputs[0]
            

            for j in range(logits.size(0)):
              probs = F.softmax(logits[j], -1)
              output_dict = {
                  # 'index': batch_size * i + j,
                  'true': b_labels[j].cpu().numpy().tolist(),
                  'pred': logits[j].argmax().item(),
                  'conf': probs.max().item(),
                  'logits': logits[j].cpu().numpy().tolist(),
                  'probs': probs.cpu().numpy().tolist(),
                  'actsent_ids'   : b_ids[j].cpu().numpy().tolist(),
                  'sent_ids'   : b_index[j].cpu().numpy().tolist(),
                  'sents' : sents[b_index[j]],
                  'words': words[b_index[j]],
                  'doc_id': doc_id[b_index[j]]
              }
              output_dicts.append(output_dict)
        y_true = [output_dict['true'] for output_dict in output_dicts]
        y_pred = [output_dict['pred'] for output_dict in output_dicts]
        y_conf = [output_dict['conf'] for output_dict in output_dicts]

        accuracy = accuracy_score(y_true, y_pred) * 100.
        fscore = f1_score(y_true, y_pred, average='macro') * 100.
        confidence = np.mean(y_conf) * 100.

        results_dict = {
            'accuracy': accuracy_score(y_true, y_pred) * 100.,
            'macro-F1': f1_score(y_true, y_pred, average='macro') * 100.,
            'confidence': np.mean(y_conf) * 100.,
        }
        #print(results_dict)
        print(classification_report(y_true,y_pred))
        print(confusion_matrix(y_true,y_pred))
        return output_dicts

In [ ]:
root_dir

'/content/gdrive/My Drive/CMED_Harvard_Shared_task//trainingdata_v3/'

In [ ]:
path=root_dir
m = Model(path,f1=1)

flag status:(1:sentence pair, 0: single sentence) 1


In [ ]:
sentences_train,entity_train,labels_train,doc_id_train=m.extract_data('train.csv')


In [ ]:
sentences_train[0:10]

In [ ]:
labels_train[0:10]

array(['Disposition ', 'Disposition ', 'Disposition ', 'Disposition ',
       'Disposition ', 'Disposition ', 'NoDisposition ', 'NoDisposition ',
       'Disposition ', 'NoDisposition '], dtype=object)

In [ ]:
print(len(sentences_train),len(entity_train),len(labels_train))

6196 6196 6196


In [ ]:

m.process_inputs(entity_train,sentences_train,labels_train)

#path_=os.path.join(root_dir1,'biobert_T')




{'Disposition ': 0, 'NoDisposition ': 1, 'Undetermined ': 2}


In [ ]:
#Train the model from scratch
m.train_save_load(path_,train=1)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Train loss: 0.5718301748184814


Epoch:  50%|█████     | 2/4 [04:05<04:05, 122.72s/it]

Train loss: 0.3582690090094645


Epoch:  75%|███████▌  | 3/4 [06:07<02:02, 122.33s/it]

Train loss: 0.2464298645860141


Epoch: 100%|██████████| 4/4 [08:09<00:00, 122.34s/it]

Train loss: 0.16854178746106083
--- 489.38185691833496 seconds ---


In [ ]:
m.train_save_load(path_,train=0)

In [ ]:
#single sentence
m.process_inputs(entity_train,sentences_train,labels_train)

path_=os.path.join(root_dir1,'biobert_T')

{'Disposition ': 0, 'NoDisposition ': 1, 'Undetermined ': 2}


In [ ]:
m.train_save_load(path_,train=1)#single sentence

In [ ]:
sentences_dev,entity_dev,labels_dev,doc_id_dev=m.extract_data('dev.csv')
act_ids=[]

#process_inputs_test(self,e,sentences,labels,act_ids,batch_size=1)
for i,sent in enumerate(sentences_dev):
  act_ids.append(i)
  




In [ ]:
m.process_inputs_test(entity_dev,sentences_dev,labels_dev,act_ids)

{'Disposition ': 0, 'NoDisposition ': 1, 'Undetermined ': 2}


In [ ]:

out=m.test_id(sentences_dev,entity_dev,doc_id_dev)

#m.compute()

              precision    recall  f1-score   support

           0       0.88      0.81      0.85       221
           1       0.90      0.97      0.93       725
           2       0.60      0.33      0.43        87

    accuracy                           0.88      1033
   macro avg       0.79      0.70      0.73      1033
weighted avg       0.87      0.88      0.87      1033

[[180  29  12]
 [ 18 700   7]
 [  7  51  29]]


In [ ]:
out[1]

{'actsent_ids': 168,
 'conf': 0.9788994193077087,
 'doc_id': '376-02',
 'logits': [-1.3231489658355713, 3.241909980773926, -1.254746913909912],
 'pred': 1,
 'probs': [0.01018962636590004, 0.9788994193077087, 0.010911006480455399],
 'sent_ids': 168,
 'sents': '(LISINOPRIL/HYDROCLOROTH...\t ',
 'true': 1,
 'words': 'LISINOPRIL/HYDROCLOROTH'}

In [ ]:
df_cmed = pd.DataFrame(out)

In [ ]:
df_cmed1=df_cmed.drop(columns=["logits","probs","sent_ids","conf"])

In [ ]:
df_cmed1.to_csv(root_dir+"/all_predictions_coarse_grained_classif_BERTpair")

In [ ]:
false=df_cmed[df_cmed["true"]!=df_cmed["pred"]]

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
false1.head()

,true,pred,actsent_ids,sents,words,doc_id
8,0,1,620,Continue depakote and lithium (had subthereapeutic level that psych thought,depakote,100-04
14,2,1,75,1. Tears and Saliva.\n,Tears,172-01
17,2,1,869,folate 1 mg,folate,131-05
18,1,0,766,and she was at that time\non lisinopril as well as her other diabetic medications.,lisinopril,302-04
26,1,2,401,and \nprednisone,prednisone,187-02


In [ ]:
false1.to_csv(root_dir+"/false_predictions_coarse_grained_classif_BERTpair")

In [ ]:
f=false1[(false1.true==2) & (false1.pred==1)]

In [ ]:
f.head()

,true,pred,actsent_ids,sents,words,doc_id
14,2,1,75,1. Tears and Saliva.\n,Tears,172-01
17,2,1,869,folate 1 mg,folate,131-05
61,2,1,647,"In the meantime continue on the Lamictal 100 mg twice a day as a \nstabilizer, Risperdal 0.25 mg per day as a mood stabilizer, the \ntestolactone 50 mg 4 times a day to increase the availability of \ntestosterone, the Clomid (clomiphene citrate) 50 mg per day to decrease \nestradiol, Levoxyl 50 mcg twice a day, Altace 10 mg twice a day regarding \nhypertension, and ibuprofen 800 mg twice a day for low back pain.",Risperdal,187-01
86,2,1,567,Proventil two puffs inhaled,Proventil,188-02
133,2,1,915,"-advair, spiriva.",spiriva,389-02
